In [69]:
ReloadProject('deep_learning')

notebook_init.py imported and reloaded
forwarded symbol: Activation
forwarded symbol: Dense
forwarded symbol: Dict
forwarded symbol: InputLayer
forwarded symbol: List
forwarded symbol: Model
forwarded symbol: Sequential
forwarded symbol: Tuple
reloaded: gym
forwarded symbol: gym
reloaded: keras
forwarded symbol: keras
reloaded: openai_wrapper
forwarded symbol: openai_wrapper
reloaded: policy_impl
forwarded symbol: policy_impl
reloaded: q_function_memoization
forwarded symbol: q_function_memoization
reloaded: q_learning
forwarded symbol: q_learning
reloaded: q_learning_impl
forwarded symbol: q_learning_impl
reloaded: q_learning_impl_v2
forwarded symbol: q_learning_impl_v2
reloaded: q_learning_v2
forwarded symbol: q_learning_v2


## Test the memoization container

In [68]:
container = q_function_memoization.TestContinuousMemoizationContainer()

expect 0.0: 0.0
expect 2.0: 2.0
expect 3.0: 3.0
expect 4.0: 4.0
expect 1: 1
expect 4.0: 4.0
expect 1: 1
expect 3.0: 3.0
expect 2: 2
expect 8.0: 8.0
expect 1: 1
expect 3.6xxx: 3.6666666666666665
expect 5: 5


## Discrete Circular World

In [70]:
STATE_ZERO_ARRAY = np.zeros(1, dtype=int)
TARGET_STATE = 5


class CircularWorld(q_learning_v2.Environment):
    
    def __init__(self):
        super().__init__(state_array_size=1, action_space_size=3)

        self.debug_verbosity = 0
        
        # action encoding
        self._action_minus = 0
        self._action_stay = 1
        self._action_plus = 2

        
    #@ Override
    def TakeAction(self, action: q_learning_v2.Action) -> q_learning_v2.Reward:
        current_state = self.GetState()
        new_state = current_state
        reward = 0
        if action == self._action_plus:
            if current_state < TARGET_STATE:
                reward = 1.0
            else:
                reward = -1.0
            new_state = current_state + 1
            if new_state == 11:
                new_state = STATE_ZERO_ARRAY
        elif action == self._action_minus:
            if current_state > TARGET_STATE:
                reward = 1.0
            else:
                reward = -1.0
            new_state = current_state - 1
            if new_state == -1:
                new_state = STATE_ZERO_ARRAY + 10
        else:
            if current_state != TARGET_STATE:
                reward = -1.0
            else:
                reward = 1.0

        self._protected_SetState(new_state)
        if self.debug_verbosity >= 1:
            print('Action %s: (%s) -> (%s), reward: %s' % (
                action, current_state, new_state, reward))
        return reward

In [72]:
%%time

env = CircularWorld()
qfunc = q_function_memoization.ContinuousMemoizationQFunction(
    env, buffer_size=40, learning_rate=0.9, discount_factor=0.95)

# Train.
# policy = policy_impl.RandomActionPolicy()
# q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=2000, debug_verbosity=0)

# Test.
policy = policy_impl.MaxValuePolicy()
q_learning_v2.Run(env=env, qfunc=qfunc, policy=policy, num_of_runs=20, debug_verbosity=10)

GET: ([0.], 0) -> 0.0
GET: ([0.], 1) -> 0.0
GET: ([0.], 2) -> 0.0
Action 0: ([0.]) -> ([10]), reward: -1.0
GET: ([10], 0) -> 0.0
GET: ([10], 1) -> 0.0
GET: ([10], 2) -> 0.0
GET: ([0.], 0) -> 0.0
SET: ([0.], 0) <- -0.9
GET: ([10], 0) -> -0.9
GET: ([10], 1) -> 0.0
GET: ([10], 2) -> 0.0
Action 1: ([10]) -> ([10]), reward: -1.0
GET: ([10], 0) -> -0.9
GET: ([10], 1) -> 0.0
GET: ([10], 2) -> 0.0
GET: ([10], 1) -> 0.0
SET: ([10], 1) <- -0.9
GET: ([10], 0) -> -0.9
GET: ([10], 1) -> -0.9
GET: ([10], 2) -> 0.0
Action 2: ([10]) -> ([0]), reward: -1.0
GET: ([0], 0) -> -0.9
GET: ([0], 1) -> -0.9
GET: ([0], 2) -> 0.0
GET: ([10], 2) -> 0.0
SET: ([10], 2) <- -0.9
GET: ([0], 0) -> -0.9
GET: ([0], 1) -> -0.9
GET: ([0], 2) -> -0.9
Action 0: ([0]) -> ([10]), reward: -1.0
GET: ([10], 0) -> -0.9
GET: ([10], 1) -> -0.9
GET: ([10], 2) -> -0.9
GET: ([0], 0) -> -0.9
SET: ([0], 0) <- -1.7595
GET: ([10], 0) -> -1.7595
GET: ([10], 1) -> -0.9
GET: ([10], 2) -> -0.9
Action 1: ([10]) -> ([10]), reward: -1.0
GET: ([10